# Detección y clasificación de vehículos por modelo en Perú.

Tanto el dataset y entrenamiento se hicieron con la ayuda de la herramienta roboflow. Donde se construyó un dataset con etiquetación manual para los tipos de vehiculos que transitan en las carreteras de Perú, incluyendo preprocesamiento y aumentación. A continuación una descripción del dataset:

The dataset includes 2531 images.
Cars are annotated in YOLO v5 PyTorch format.

The following pre-processing was applied to each image:
* Auto-orientation of pixel data (with EXIF-orientation stripping)

The following augmentation was applied to create 3 versions of each source image:
* 50% probability of horizontal flip
* Randomly crop between 0 and 50 percent of the image
* Random shear of between -15° to +15° horizontally and -15° to +15° vertically
* Random exposure adjustment of between -41 and +41 percent
* Salt and pepper noise was applied to 5 percent of pixels

Para hacer la detección importaremos el modelo con una API creada por roboflow para optimizar el funcionamiento y memoria.



In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## Roboflow API

Importamos la API usando la biblioteca roboflow que procedemos a descargar primeramente

In [ ]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0


Como tal la detección tiene la misma función a la implementación de YOLOv5 en python. Veamos una prueba con una imagen extraida de la data (videos) original.

In [ ]:
#Importamos API

from roboflow import Roboflow
rf = Roboflow(api_key="API KEY")
project = rf.workspace().project("NAME PROJECT")
# Modelo detección
model = project.version(1).model

loading Roboflow workspace...
loading Roboflow project...


Hacemos la predicción unicamente mostrando la etiqueta obtenida.

In [ ]:
path_image = "Path a imagen de prueba.png"

# Hacer la predicción
print(model.predict(path_image, confidence=40, overlap=30).json())

{'predictions': [{'x': 1357.5, 'y': 449.0, 'width': 261.0, 'height': 84.0, 'confidence': 0.6129214763641357, 'class': 'auto', 'class_id': 11, 'detection_id': '76861349-286b-4c7b-aa4c-90ec69f32e6c', 'image_path': '/content/drive/MyDrive/PY_PROJECTS/ALGORITHMICA/ASESORIAS/CARLOS-AUTOS/ENTRENAMIENTO_FINAL/Dataset/prueba2.png', 'prediction_type': 'ObjectDetectionModel'}, {'x': 558.0, 'y': 435.0, 'width': 322.0, 'height': 122.0, 'confidence': 0.4627867639064789, 'class': 'combi', 'class_id': 13, 'detection_id': 'b013a92a-012b-4466-a5b5-d8325730a709', 'image_path': '/content/drive/MyDrive/PY_PROJECTS/ALGORITHMICA/ASESORIAS/CARLOS-AUTOS/ENTRENAMIENTO_FINAL/Dataset/prueba2.png', 'prediction_type': 'ObjectDetectionModel'}], 'image': {'width': '1920', 'height': '839'}}


Predicción mostrando la imagen con su bounding box

In [ ]:
from google.colab import files

path_image = "Path a imagen de prueba.png"

# visualize your prediction
model.predict(path_image, confidence=40, overlap=30).save("prediction.jpg")
files.download("prediction.jpg")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Detección

Llamamos a la detección para un video de prueba de titulo 'cut_1.mp4' y la hacemos con la ayuda de OpenCv para poder extraer frame por frame. A la predicción por frame la guardamos en una lista llamada data.

In [ ]:
import cv2

# Cambiar aquí el path al video a detectar
cap = cv2.VideoCapture('Detección a video de trafico.mp4')
data = []
frame_times = []  # Lista para almacenar los tiempos de cada frame
i = 0

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    # Obtener los vectores de predicción de los objetos detectados
    pred_vectors = model.predict(frame, confidence=40, overlap=30).json()

    # Guardar todos los datos en una lista, donde cada elemento tiene el número de listas correspondiente al número de objetos detectados
    data.append(pred_vectors)

    # Obtener el tiempo actual en segundos y agregarlo a la lista
    current_time = i / cap.get(cv2.CAP_PROP_FPS)
    frame_times.append(current_time)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    i += 1

cap.release()
cv2.destroyAllWindows()

# Imprimir los tiempos de cada frame
#for frame_num, time in enumerate(frame_times):
#    print(f"Frame {frame_num + 1} - Tiempo: {time:.2f} s")



Pasamos de tener un dataset en fomrato json a lista de listas por vehiculo detectado.

In [ ]:
formatted_data = []
aux = []

for item in data:
    predictions = item['predictions']
    aux = []
    for prediction in predictions:
        x = prediction['x']
        y = prediction['y']
        width = prediction['width']
        height = prediction['height']
        confidence = prediction['confidence']
        class_name = prediction['class']

        prediction = [x, y, width, height, confidence, 1, class_name]
        aux.append(prediction)

    formatted_data.append(aux)

print(formatted_data)


[[[420.5, 417.5, 213.0, 67.0, 0.8823843002319336, 1, 'camioneta']], [[430.5, 417.0, 209.0, 68.0, 0.6340402364730835, 1, 'auto']], [[443.5, 416.5, 211.0, 67.0, 0.8700859546661377, 1, 'camioneta']], [[455.0, 416.0, 210.0, 68.0, 0.7663540840148926, 1, 'camioneta']], [[464.5, 416.0, 209.0, 70.0, 0.8561602830886841, 1, 'auto']], [[476.0, 415.0, 214.0, 68.0, 0.8957734107971191, 1, 'auto']], [[488.5, 415.0, 207.0, 68.0, 0.8991923928260803, 1, 'auto']], [[501.0, 414.5, 206.0, 67.0, 0.8871128559112549, 1, 'auto']], [[512.5, 415.0, 207.0, 66.0, 0.894039511680603, 1, 'auto']], [[524.0, 414.5, 202.0, 65.0, 0.9032331705093384, 1, 'auto']], [[534.0, 415.0, 202.0, 64.0, 0.889866828918457, 1, 'auto']], [[546.5, 414.0, 203.0, 66.0, 0.8750429153442383, 1, 'auto']], [[555.5, 414.0, 201.0, 66.0, 0.8758805394172668, 1, 'auto'], [26.0, 427.5, 52.0, 77.0, 0.4728562831878662, 1, 'combi']], [[570.0, 414.0, 198.0, 64.0, 0.8860189914703369, 1, 'auto'], [33.5, 427.0, 67.0, 82.0, 0.5167100429534912, 1, 'combi']], 

Guardamos la detección en una lista para no tener que correr todo cada vez

In [ ]:
import pickle
from google.colab import files

# Guardar la lista en un archivo usando pickle
archivo_guardado = "data.pkl"
with open(archivo_guardado, 'wb') as archivo:
    pickle.dump(formatted_data, archivo)

print("Lista guardada en el archivo:", archivo_guardado)
files.download(archivo_guardado)

Lista guardada en el archivo: data.pkl


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Analisis de Vehiculos

Una vez hecha la detección procedemos a analizar si el auto se dirige de norte a sur o de sur a norte y la hora de la detección.

In [ ]:
import pickle
from google.colab import files

# Cargar la lista desde el archivo usando pickle
archivo_cargado = "Modelo entrenado model.pkl"
with open(archivo_cargado, 'rb') as archivo:
    data_cargada = pickle.load(archivo)

print("Lista cargada desde el archivo:", data_cargada)

Lista cargada desde el archivo: [[[420.5, 417.5, 213.0, 67.0, 0.8823843002319336, 1, 'camioneta']], [[430.5, 417.0, 209.0, 68.0, 0.6340402364730835, 1, 'auto']], [[443.5, 416.5, 211.0, 67.0, 0.8700859546661377, 1, 'camioneta']], [[455.0, 416.0, 210.0, 68.0, 0.7663540840148926, 1, 'camioneta']], [[464.5, 416.0, 209.0, 70.0, 0.8561602830886841, 1, 'auto']], [[476.0, 415.0, 214.0, 68.0, 0.8957734107971191, 1, 'auto']], [[488.5, 415.0, 207.0, 68.0, 0.8991923928260803, 1, 'auto']], [[501.0, 414.5, 206.0, 67.0, 0.8871128559112549, 1, 'auto']], [[512.5, 415.0, 207.0, 66.0, 0.894039511680603, 1, 'auto']], [[524.0, 414.5, 202.0, 65.0, 0.9032331705093384, 1, 'auto']], [[534.0, 415.0, 202.0, 64.0, 0.889866828918457, 1, 'auto']], [[546.5, 414.0, 203.0, 66.0, 0.8750429153442383, 1, 'auto']], [[555.5, 414.0, 201.0, 66.0, 0.8758805394172668, 1, 'auto'], [26.0, 427.5, 52.0, 77.0, 0.4728562831878662, 1, 'combi']], [[570.0, 414.0, 198.0, 64.0, 0.8860189914703369, 1, 'auto'], [33.5, 427.0, 67.0, 82.0, 0.

La siguiente rutina parte de un frame, que es una lista con los vectores etiqueta de cada objeto detectado en dicho frame y declarada cierta ventana de detección se encarga de ver cuantos vehiculos pasan por esta ventana con el fin de saber cuantos vehiculos individuales transitan por la carretera sin dar oportunidad a que se repita el mismo en un solo viaje.

La rutina devuelve el numero de vehiculos detectados por frame, se pueden descomentar los print para ver si no se detecta ninguno en el frame o cuantos vehiculos se detecatorn por frame y el numero de llantas

In [ ]:
#Rutina para detectar a partir de un frame (lista de python) qcuantos vehiculos pasan a travez de una cierta zona delimitada (en x)
def deteccion_con_ventana_iteracion(frame_work):

  # Rutina oculta

In [ ]:
#Rutina para detectar a partir de un frame (lista de python) qcuantos vehiculos pasan a travez de una cierta zona delimitada (en x)
def deteccion_con_ventana(frame_work):

  # Rutina oculta

##Conteo de vehiculos en transito por fragmento de video

In [ ]:
#Variables de conteo
i = 0
num_total = 0

#Recorremos todo el dataset para contar cuantos vehiculos han transitado sin detección
for frame in data_cargada:
  num_vehi, index_vehicles = deteccion_con_ventana(frame)

  if (num_vehi != 0):
    for j in range(num_vehi):
      num_vehi2, index_vehicles2 = deteccion_con_ventana_iteracion(data_cargada[i+1])

      if((index_vehicles2[j][0]-index_vehicles[j][0]) > 0):
        print('----------------------------------------------')
        print(f"Se detectaron {num_vehi} vehiculos:")
        for ind in index_vehicles:
          print(f"Se detectó un tipo de vehiculo '{ind[1]}' en el segundo {frame_times[i]} y va de izquierda a derecha")

      if((index_vehicles2[j][0]-index_vehicles[j][0]) < 0):
        print('----------------------------------------------')
        print(f"Se detectaron {num_vehi} vehiculos:")
        for ind in index_vehicles:
          print(f"Se detectó un tipo de vehiculo '{ind[1]}' en el segundo {frame_times[i]} y va de derecha a izquierda")

  num_total = num_total + num_vehi
  i = i+1


print('\n\n')
print("En el fragmento de video analizado se detectó el transito de un total de", num_total, "vehiculos")

----------------------------------------------
Se detectaron 1 vehiculos:
Se detectó un tipo de vehiculo 'auto' en el segundo 0.4334778259419807 y va de izquierda a derecha
----------------------------------------------
Se detectaron 1 vehiculos:
Se detectó un tipo de vehiculo 'auto' en el segundo 2.534178059353118 y va de izquierda a derecha
----------------------------------------------
Se detectaron 1 vehiculos:
Se detectó un tipo de vehiculo 'combi' en el segundo 3.9346448816272095 y va de izquierda a derecha
----------------------------------------------
Se detectaron 1 vehiculos:
Se detectó un tipo de vehiculo 'auto' en el segundo 15.005001667222409 y va de izquierda a derecha
----------------------------------------------
Se detectaron 1 vehiculos:
Se detectó un tipo de vehiculo 'combi' en el segundo 77.85928642880961 y va de izquierda a derecha
----------------------------------------------
Se detectaron 1 vehiculos:
Se detectó un tipo de vehiculo 'auto' en el segundo 83.361120